In [36]:
import pandas as pd
import os
import requests
import io
from urllib.request import urlopen
import zipfile
import warnings
import unidecode
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
from urllib.request import urlopen
import zipfile
import difflib
import basedosdados as bd

warnings.filterwarnings("ignore")

In [4]:
for url in range(2014, 2025):
    url = f"https://portaldatransparencia.gov.br/download-de-dados/orcamento-despesa/{url}"
    if requests.get(url).status_code == 200:

        r = urlopen(url)
        z = zipfile.ZipFile(io.BytesIO(r.read()))
        z.extractall("/home/tricktx/dados/br_cgu_orcamento_publico")

In [37]:
def to_partitions(
    data: pd.DataFrame,
    partition_columns: List[str],
    savepath: str,
    file_type: str = "csv",
):
    """Save data in to hive patitions schema, given a dataframe and a list of partition columns.
    Args:
        data (pandas.core.frame.DataFrame): Dataframe to be partitioned.
        partition_columns (list): List of columns to be used as partitions.
        savepath (str, pathlib.PosixPath): folder path to save the partitions.
        file_type (str): default to csv. Accepts parquet.
    Exemple:
        data = {
            "ano": [2020, 2021, 2020, 2021, 2020, 2021, 2021,2025],
            "mes": [1, 2, 3, 4, 5, 6, 6,9],
            "sigla_uf": ["SP", "SP", "RJ", "RJ", "PR", "PR", "PR","PR"],
            "dado": ["a", "b", "c", "d", "e", "f", "g",'h'],
        }
        to_partitions(
            data=pd.DataFrame(data),
            partition_columns=['ano','mes','sigla_uf'],
            savepath='partitions/',
        )
    """

    if isinstance(data, (pd.core.frame.DataFrame)):
        savepath = Path(savepath)
        # create unique combinations between partition columns
        unique_combinations = (
            data[partition_columns]
            # .astype(str)
            .drop_duplicates(subset=partition_columns).to_dict(orient="records")
        )

        for filter_combination in unique_combinations:
            patitions_values = [
                f"{partition}={value}"
                for partition, value in filter_combination.items()
            ]

            # get filtered data
            df_filter = data.loc[
                data[filter_combination.keys()]
                .isin(filter_combination.values())
                .all(axis=1),
                :,
            ]
            df_filter = df_filter.drop(columns=partition_columns)

            # create folder tree
            filter_save_path = Path(savepath / "/".join(patitions_values))
            filter_save_path.mkdir(parents=True, exist_ok=True)

            if file_type == "csv":
                # append data to csv
                file_filter_save_path = Path(filter_save_path) / "data.csv"
                df_filter.to_csv(
                    file_filter_save_path,
                    sep=",",
                    encoding="utf-8",
                    na_rep="",
                    index=False,
                    mode="a",
                    header=not file_filter_save_path.exists(),
                )
            elif file_type == "parquet":
                # append data to parquet
                file_filter_save_path = Path(filter_save_path) / "data.parquet"
                df_filter.to_parquet(
                    file_filter_save_path, index=False, compression="gzip"
                )
    else:
        raise BaseException("Data need to be a pandas DataFrame")

In [38]:
for path in range(2014, 2025):
    path = f"/home/tricktx/dados_basedosdados/br_cgu_orcamento_publico/{path}_OrcamentoDespesa.csv"
    print(path)
    df = pd.read_csv(
        path,
        sep=";",
        encoding="latin1",
    )

    df.columns = [unidecode.unidecode(col) for col in df.columns]
    df.columns = [col.replace(" ", "_").lower() for col in df.columns]
    df.columns = [col.replace("exercicio", "ano_exercicio") for col in df.columns]
    df.columns = [col.replace("%_realizado_do_orcamento_(com_relacao_ao_orcamento_atualizado)", "porcentagem_realizado_orcamento") for col in df.columns]
    df['porcentagem_realizado_orcamento'] = df['porcentagem_realizado_orcamento'].str.replace("%", "")

    valores = [
        "orcamento_inicial_(r$)",
        "orcamento_atualizado_(r$)",
        "orcamento_empenhado_(r$)",
        "orcamento_realizado_(r$)",
        "porcentagem_realizado_orcamento"
    ]


    for x in valores:
        df[x] = df[x].str.replace(",", ".").astype(float)
    df.rename(columns=lambda x: x.replace("_(r$)", ""), inplace=True)
    print(df.columns)
    to_partitions(
        data=df,
        partition_columns=["ano_exercicio"],
        savepath="/home/tricktx/dados/br_cgu_orcamento_publico/partitions/",
    )

/home/tricktx/dados_basedosdados/br_cgu_orcamento_publico/2014_OrcamentoDespesa.csv
Index(['ano_exercicio', 'codigo_orgao_superior', 'nome_orgao_superior',
       'codigo_orgao_subordinado', 'nome_orgao_subordinado',
       'codigo_unidade_orcamentaria', 'nome_unidade_orcamentaria',
       'codigo_funcao', 'nome_funcao', 'codigo_subfuncao', 'nome_subfuncao',
       'codigo_programa_orcamentario', 'nome_programa_orcamentario',
       'codigo_acao', 'nome_acao', 'codigo_categoria_economica',
       'nome_categoria_economica', 'codigo_grupo_de_despesa',
       'nome_grupo_de_despesa', 'codigo_elemento_de_despesa',
       'nome_elemento_de_despesa', 'orcamento_inicial', 'orcamento_atualizado',
       'orcamento_empenhado', 'orcamento_realizado',
       'porcentagem_realizado_orcamento'],
      dtype='object')


ModuleNotFoundError: No module named 'pandas.core.methods.to_dict'

In [8]:
df.columns = [unidecode.unidecode(col) for col in df.columns]
df.columns = [col.replace(" ", "_").lower() for col in df.columns]

In [13]:
df = pd.read_csv(
    "/home/tricktx/dados_basedosdados/br_cgu_orcamento_publico/2023_OrcamentoDespesa.csv",
    sep=";",
    encoding="latin1",
)

df.columns = [unidecode.unidecode(col) for col in df.columns]
df.columns = [col.replace(" ", "_").lower() for col in df.columns]

df.columns = [col.replace("%_realizado_do_orcamento_(com_relacao_ao_orcamento_atualizado)", "porcentagem_realizado_orcamento") for col in df.columns]
df['porcentagem_realizado_orcamento'] = df['porcentagem_realizado_orcamento'].str.replace("%", "")

valores = [
    "orcamento_inicial_(r$)",
    "orcamento_atualizado_(r$)",
    "orcamento_empenhado_(r$)",
    "orcamento_realizado_(r$)",
    "porcentagem_realizado_orcamento"
]


for x in valores:
    df[x] = df[x].str.replace(",", ".").astype(float)
df.rename(columns=lambda x: x.replace("_(r$)", ""), inplace=True)

In [29]:
df = pd.read_csv(
    "/home/tricktx/dados_basedosdados/br_cgu_orcamento_publico/2023_OrcamentoDespesa.csv",
    sep=";",
    encoding="latin1",
)

df.columns = [unidecode.unidecode(col) for col in df.columns]
df.columns = [col.replace(" ", "_").lower() for col in df.columns]

In [31]:
df.columns = [col.replace("exercicio", "ano_exercicio") for col in df.columns]

In [35]:
df['ano_exercicio'].value_counts()

ano_exercicio
2014    28641
Name: count, dtype: int64